In [368]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pandas_profiling import ProfileReport
import pickle
import os
import datetime
from scipy.stats import zscore

In [2]:
# Upload and clean
df, _, _ = pickle.load(open('C:/Users/Brayden/Desktop/LocScrapes/Indian Creek (Full).pkl', 'rb'))
df.drop_duplicates(subset='URL', inplace=True)

In [ ]:
# Prefab EDA
profile = ProfileReport(df)
profile.to_widgets()

In [3]:
# Number of Ticks Exploration
pd.to_numeric(df['Num Ticks'].fillna(0, inplace=True), downcast='integer')

col=df['Num Ticks']
yperc = np.percentile(col, range(0,100))

fig1 = px.histogram(col, nbins=500, marginal='box', width=800)
fig1.update_layout(showlegend=False, title_x=0.5)
fig1.update_xaxes(title="Number of Ticks", row=1,col=1)
fig1
col.describe()
fig2 = px.line(x=yperc, y=range(0,100), title='Number of Ticks Percentile Plot', width=800)
fig2.update_layout(title_x=0.5)
fig2.update_xaxes(title="Number of Ticks", range=[-10,1000])
fig2.update_yaxes(title="Percentile (%)")
fig2.update_traces(line_width=4)
fig2

In [4]:
# Percent of climbs responsible for percent of ticks analysis
pd.to_numeric(df['Num Ticks'].fillna(0, inplace=True), downcast='integer')
df_sorted = df.sort_values('Num Ticks', ascending=False)['Num Ticks']
sum_ticks = df['Num Ticks'].sum()
num_climbs = len(df.index)
df_sorted_sums = []
for i in range(0,num_climbs):
    df_sorted_sums.append(df_sorted.nlargest(i).sum())
df_sorted_sums = (df_sorted_sums/sum_ticks)*100
df_perc_of_climbs = np.round_((np.array([*range(0,num_climbs)])/num_climbs)*100, 1)

fig = px.line(x=df_perc_of_climbs, y=df_sorted_sums, title="Percent of Climbs Responsible for Percent of Ticks", width=800, height=400)
fig.update_yaxes(range = [0,100], title="Percent of Ticks (%)")
fig.update_xaxes(range = [0,100], title="Percent of Climbs (%)")
fig.update_layout(title_x=0.5)
fig

fig = px.line(x=[*range(0,num_climbs)], y=df_sorted_sums, title="N Most Ticked Routes Vs. Percent of Ticks", width=800, height=400)
fig.update_yaxes(range = [0,100], title="Percent of Ticks (%)")
fig.update_xaxes(range = [0,500], title="N Highest Ticked Routes")
fig.update_layout(title_x=0.5)
fig

In [49]:
# Top climbed analysis
top_climbed = df.sort_values('Num Ticks', ascending=False)[0:29]
plt.rcParams['figure.figsize']=(30,5)
fig = px.bar(top_climbed, x='Route', y='Num Ticks', title="Top 30 Routes Tick Counts", text_auto=True, width=1200)
fig.update_xaxes(title='Route Name')
fig.update_yaxes(title='Number of Ticks')
fig.update_layout(title_x=0.5)

In [73]:
list=['5.10',
'5.10',
'5.9+',
'5.10-',
'5.10-',
'5.8+',
'5.11-',
'5.8+',
'5.10',
'5.10-',
'5.10',
'5.10+',
'5.11+',
'5.7',
'5.10',
'5.9',
'5.10',
'5.12-',
'5.11-',
'5.10',
'5.10',
'5.11+',
'5.9',
'5.10-',
'5.11',
'5.10',
'5.9',
'5.9+',
'5.10']

fig = px.histogram(list, title='Grade Distribution of Top 30 Routes by Number of Ticks', width=1200)
fig.update_xaxes(type='category', categoryorder='array', categoryarray=['5.7', '5.8', '5.8+', '5.9', '5.9+', '5.10-', '5.10', '5.10+', '5.11-', '5.11', '5.11+'], title='Grade')
fig.update_layout(title_x=0.5, showlegend=False)

In [16]:
# Sum ticks by area
df_loc_traf = df.groupby('Location')['Num Ticks'].sum().sort_values(ascending=False).to_frame().reset_index()
df_loc_traf['Location'] = df_loc_traf['Location'].apply(lambda x: x.split('>')[0])
fig = px.bar(df_loc_traf.iloc[0:30], x='Location', y='Num Ticks', title="Top 30 Crags Tick Sums", text_auto=True, width=1200)
fig.update_xaxes(title='Crag Name')
fig.update_yaxes(title='Sum Number of Ticks')
fig.update_layout(title_x=0.5)

In [7]:
df['Lead Ratio'].fillna(0, inplace=True)
dfs = df[(df['Num Ticks']>=100)]

col=dfs['Lead Ratio']

fig1 = px.histogram(col, marginal='box', width=800, title="Lead Ratio (Number of Ticks > 100)")
fig1.update_layout(showlegend=False, title_x=0.5)
fig1.update_xaxes(title="Lead Ratio", row=1,col=1, range=[0,1])

In [8]:
dfs[dfs['Lead Ratio']<0.4].sort_values('Lead Ratio', ascending=True)[['Route', 'Location' ,'URL', 'Rating', 'Num Ticks', 'Lead Ratio']].style.set_properties(subset='URL', **{'width':'300px'})

,Route,Location,URL,Rating,Num Ticks,Lead Ratio
1028,Pringles,Supercrack Buttress > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/105717427/pringles,5.11+,143.000000,0.064815
900,Naked and the Dead Variation,Donnelly Canyon > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/105943086/naked-and-the-dead-variation,5.11,221.000000,0.068027
1203,Inflictor,Broken Tooth > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/106123505/inflictor,5.12-,112.000000,0.173913
278,Town,4X4 > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/105718396/town,5.10d,225.000000,0.267974
491,Dirty Woman,The Wall > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/107385681/dirty-woman,5.10,113.000000,0.270588
922,New World Order,Battle of the Bulge Buttress > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/106155187/new-world-order,5.11b/c,112.000000,0.289474
819,Mariposa,Habitado > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/116668339/mariposa,5.10-,212.000000,0.338983
631,(Unknown),Selfish Wall > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/109590225/unknown,5.9,180.000000,0.353741
94,Unnamed 5.10 Flake,Blue Gramma Cliff > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/106141573/unnamed-510-flake,5.10,118.000000,0.356164
115,The Fat Farm,Scarface > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/106069249/the-fat-farm,5.10,188.000000,0.368421


In [9]:
col=dfs['Repeat Sender Ratio']

fig1 = px.histogram(col, marginal='box', width=800, title="Repeat Sender Ratio (Number of Ticks > 100)")
fig1.update_layout(showlegend=False, title_x=0.5)
fig1.update_xaxes(title="Lead Ratio", row=1,col=1, range=_)

In [329]:
# Create repeat sender sum value not given by original analysis
df['Repeat Sender Sum'] = df['Route Ticks'].apply(lambda x: 
            x[x["Lead Style"].isin(["Onsight", "Flash", "Redpoint", "Pinkpoint", "Send"])]
            .groupby("Username")["Lead Style"]
            .count()
            .sum()
        )

In [330]:
col=df['Repeat Sender Sum']

fig1 = px.histogram(col, marginal='box', width=800, title="Repeat Sender Sum (Number of Ticks > 100)")
fig1.update_layout(showlegend=False, title_x=0.5)
fig1.update_xaxes(title="Lead Ratio", row=1,col=1, range=_)

In [331]:
df.sort_values('Repeat Sender Sum', ascending=False)[0:30][['Route', 'Repeat Sender Sum']]
# Most of these are repeats of top ticked climbs, with notable exceptions of
# Dr. Carl
# Jupiter Crack
# Lady Pillar
# Lightning Bolt Crack
# Unnamed 5.10 LF flake in deep corner

,Route,Repeat Sender Sum
354,Blue Sun,714
149,Incredible Hand Crack,655
148,Supercrack of the Desert,654
141,Twin Cracks,623
3,Chocolate Corner,587
4,Generic Crack,581
1,Binou's Crack,539
135,Scarface,417
114,Wavy Gravy,332
236,South Face,325


In [332]:
df['Normalized Repeat Sender Sum'] = df['Repeat Sender Sum'] / df['Num Ticks']
fig = px.histogram(df[df['Num Ticks'] > 100].sort_values('Normalized Repeat Sender Sum', ascending=False)['Normalized Repeat Sender Sum'], marginal='box', title='Normalized Repeat Sender Sum (Number of Ticks > 100)', width=1200)
fig.update_layout(showlegend=False, title_x=0.5)
fig.update_xaxes(title="Normalized Repeat Sender Sum", row=1,col=1, range=_)

In [74]:
df[df['Num Ticks'] > 100].sort_values('Normalized Repeat Sender Sum', ascending=False)['Normalized Repeat Sender Sum'].describe()

count    288.000000
mean       0.224352
std        0.084066
min        0.026786
25%        0.172218
50%        0.221853
75%        0.272288
max        0.517544
Name: Normalized Repeat Sender Sum, dtype: float64

In [48]:
fig = px.histogram(df[(df['Num Ticks'] > 100) & ~(df['Rating']=='5.10c')].sort_values('Normalized Repeat Sender Sum', ascending=False)[0:30]['Rating'], title='Top 30 Normalized Repeat Sender Sum Grades (Number of Ticks > 100)', width=1200)
fig.update_layout(showlegend=False, title_x=0.5)
fig.update_xaxes(title="Grade", row=1,col=1, range=_, type='category', categoryorder='array', categoryarray=['5.8', '5.8+', '5.9', '5.9+', '5.10-', '5.10', '5.10+', '5.11'])

In [ ]:
df[(df['Num Ticks'] > 100) & (df['Normalized Repeat Sender Sum']>=0.3)].sort_values('Normalized Repeat Sender Sum', ascending=False)

In [ ]:
dfs.sort_values('Repeat Sender Ratio', ascending=False)[0:30][['Route', 'Repeat Sender Ratio']]
# Climbs that are normalized slightly, these can be thought of as underground favorite
# 1 to 4 crack
# 24 Unknown
# Act or React
# Breakfast Social
# Cactus Flower
# Crack Slabbath
# Horse Crack
# Kelley Route 1
# Low Cholesterol
# Mariposa
# More Than One Way
# Pigs On The Wing
# Short and Stupid
# Spam
# Tag Team
# The Ooze
# The Thing
# Think Pink
# Unnamed
# Unnamed 10+ (steep thin hands dihedral to pod)
# Unnamed 9+ (big hands dihedral)

In [266]:
df, _, _ = pickle.load(open('C:/Users/Brayden/Desktop/LocScrapes/Indian Creek (Full).pkl', 'rb'))
df.drop_duplicates(subset='URL', inplace=True)

df.sort_values('Num Ticks', ascending=False, inplace=True)
df_agg=pd.DataFrame(index=range(2005,2023))
for i in range(0,len(df[df['Num Ticks']>100].index)):
    subdf = df.iloc[i]['Route Ticks']
    tick_sum_yearly = subdf.groupby(subdf.Date.dt.year)['Pitches Ticked'].sum()
    tick_sum_yearly = tick_sum_yearly[~(tick_sum_yearly.index<2005) & ~(tick_sum_yearly.index>=datetime.date.today().year)] # MP was founded in 2005, so we ignore historical ticks before that time, and we ignore the current year as it is still in progress
    if tick_sum_yearly.empty:
        tick_velo_mean = 0
    else:
        tick_sum_yearly = tick_sum_yearly.reindex(np.arange(tick_sum_yearly.index.min(), tick_sum_yearly.index.max()+1)).fillna(0) # Fill missing years with 0
        tick_sum_yearly = tick_sum_yearly.astype(int) # Previous step cast values to float, we recast to int
    tick_sum_yearly.rename(df.iloc[i]['Route'], inplace=True)
    df_agg = df_agg.merge(tick_sum_yearly, how='left', left_index=True, right_index=True)
df_agg.fillna(0, inplace=True)


In [268]:
fig = px.line(df_agg.iloc[:,range(1,100,10)], title='YoY Difference in Sum Ticks For Select Routes', height=600)
fig.update_layout(title_x=0.5, hovermode='x')
fig.update_xaxes(title='Year')
fig.update_yaxes(title='YoY Diff in Sum Ticks')
fig.write_html("YoY_Sum_Ticks.html")
fig

In [269]:
df_agg_m = pd.melt(df_agg.diff().T)
fig = px.box(df_agg_m, x='variable', y='value', title='YoY Difference in Sum Ticks For All Routes w/ >100 Ticks')
fig.update_layout(title_x=0.5)
fig.update_xaxes(title='Year', tickmode='linear')
fig.update_yaxes(title='YoY Diff in Sum Ticks')
fig.write_html("YoY_Sum_Ticks_Box.html")
fig

In [280]:
fig = px.bar(df_agg.mean(axis=1), title='Mean YoY Difference in Sum Ticks For All Routes w/ >100 Ticks')
fig.update_layout(title_x=0.5, showlegend=False)
fig.update_xaxes(title='Year', tickmode='linear')
fig.update_yaxes(title='Mean YoY Diff in Sum Ticks')
fig.write_html("Mean_YoY_Sum_Ticks_Box.html")
fig

In [282]:
def tick_avg_velo(subdf):
    tick_sum_yearly = subdf.groupby(subdf.Date.dt.year)['Pitches Ticked'].sum()
    # tick_sum_yearly = tick_sum_yearly[~(tick_sum_yearly.index<2005) & ~(tick_sum_yearly.index>=datetime.date.today().year)] # MP was founded in 2005, so we ignore historical ticks before that time, and we ignore the current year as it is still in progress
    tick_sum_yearly = tick_sum_yearly[~(tick_sum_yearly.index>=datetime.date.today().year) & (tick_sum_yearly.index>=datetime.date.today().year-6)]
    if tick_sum_yearly.empty:
        tick_velo_mean = 0
    else:
        tick_sum_yearly = tick_sum_yearly.reindex(np.arange(tick_sum_yearly.index.min(), tick_sum_yearly.index.max()+1)).fillna(0) # Fill missing years with 0
        tick_sum_yearly = tick_sum_yearly.astype(int) # Previous step cast values to float, we recast to int
        tick_velo_mean = tick_sum_yearly.diff().mean()
    return tick_velo_mean

df, _, _ = pickle.load(open('C:/Users/Brayden/Desktop/LocScrapes/Indian Creek (Full).pkl', 'rb'))
df.drop_duplicates(subset='URL', inplace=True)
df['Avg Tick Velo'] = df.apply(lambda x: tick_avg_velo(x['Route Ticks']), axis=1)

In [356]:
df = df[df['Num Ticks'] >= 100]
df['Norm Avg Tick Velo'] = df['Avg Tick Velo'] / df['Num Ticks']
df.sort_values('Norm Avg Tick Velo', ascending=False)[0:30]
fig = px.histogram(df['Norm Avg Tick Velo'], marginal='box', title='<b>Normalized Average 5yr Tick Velocity</b><br>Number of Ticks > 100')
fig.update_layout(showlegend=False, title_x=0.5)
fig.update_xaxes(title="Normalized Average 5yr Tick Velocity", row=1,col=1, range=_)
fig.update_yaxes(title='Value', row=1,col=1)
fig.write_html("Norm_Average5yr_Tick_Velocity.html")
fig

In [302]:
pd.set_option('display.max_colwidth', None)
df.sort_values('Norm Avg Tick Velo', ascending=False)[0:30][['Route', 'Location', 'URL', 'Rating', 'Norm Avg Tick Velo']]

,Route,Location,URL,Rating,Norm Avg Tick Velo
815,Unknown 5.8,Habitado > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/116668323/unknown-58,5.8+,0.166172
819,Mariposa,Habitado > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/116668339/mariposa,5.10-,0.146226
838,Zits,Trick or Treat Wall > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/116707833/zits,5.10,0.144231
816,Lightning Bolt Crack,Habitado > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/116177998/lightning-bolt-crack,5.9,0.119307
1526,Horse Crack,Trick or Treat Wall > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/116707890/horse-crack,5.11,0.116352
823,Unnamed 5.10,Habitado > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/117925164/unnamed-510,5.10,0.103516
678,Fission Quest,Sinbad Wall aka Nuclear Wall > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/116134631/fission-quest,5.10,0.092391
685,Nuclear Fingers,Sinbad Wall aka Nuclear Wall > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/116134602/nuclear-fingers,5.11-,0.088710
669,Meltdown,Sinbad Wall aka Nuclear Wall > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/116093642/meltdown,5.9+,0.080827
675,Tiffany's Mall Tour,Sinbad Wall aka Nuclear Wall > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/107658941/tiffanys-mall-tour,5.10,0.078125


In [ ]:
df_cumz = df[df['Num Ticks']>100][['Num Ticks', 'Lead Ratio', 'Normalized Repeat Sender Sum', 'Norm Avg Tick Velo']].apply(zscore)
df['C Score'] = 0.4*df_cumz['Num Ticks'] + -0.15*df_cumz['Lead Ratio'] + 0.15*df_cumz['Normalized Repeat Sender Sum'] + 0.3*df_cumz['Norm Avg Tick Velo']
df.sort_values('C Score', ascending=False)[['Route', 'Location', 'URL', 'Rating', 'Num Ticks', 'Norm Avg Tick Velo', 'Lead Ratio', 'Normalized Repeat Sender Sum', 'CZ Score']][0:60]

In [369]:
df['CZ Score'] = zscore(df['C Score'], nan_policy='omit')
df.sort_values('CZ Score', ascending=False)[0:60]

,Route,Location,URL,Avg Stars,Your Stars,Route Type,Original Rating,Rating,Pitches,Length,...,OS Ratio,Repeat Sender Ratio,Mean Attempts To RP,Tick Counts,Avg Tick Velo,Norm Avg Tick Velo,C Score,Repeat Sender Sum,Normalized Repeat Sender Sum,CZ Score
149,Incredible Hand Crack,Supercrack Buttress > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/105717367/incredible-hand-crack,3.8,NaN,Trad,5.10,5.10,1,100.0,...,0.513678,1.046326,1.591241,Lead 1178 TR 517 Onsight 404 Fell/Hung 323 Redpoint 153 Follow 110 Flash 103 Solo 4 Pinkpoint 4 Send 0 Attempt 0 dtype: int64,24.000000,0.008191,2.265847,655,0.223549,4.545311
148,Supercrack of the Desert,Supercrack Buttress > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/105717364/supercrack-of-the-desert,3.8,NaN,Trad,5.10,5.10,3,70.0,...,0.573626,1.068627,1.471154,Lead 1087 TR 544 Onsight 407 Fell/Hung 252 Redpoint 130 Flash 115 Follow 78 Pinkpoint 6 Solo 2 Send 0 Attempt 0 dtype: int64,33.000000,0.011892,2.204033,654,0.235676,4.421312
3,Chocolate Corner,Donnelly Canyon > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/105717349/chocolate-corner,3.0,NaN,Trad,5.9+,5.9+,1,50.0,...,0.555152,1.057658,1.685185,Lead 1064 TR 470 Onsight 378 Fell/Hung 234 Redpoint 130 Follow 86 Flash 80 Pinkpoint 3 Solo 2 Send 0 Attempt 0 dtype: int64,31.000000,0.012020,1.969974,587,0.227608,3.951787
4,Generic Crack,Donnelly Canyon > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/105717328/generic-crack,3.4,NaN,Trad,5.10-,5.10-,1,120.0,...,0.620205,1.068015,1.693333,Lead 945 TR 477 Onsight 397 Fell/Hung 195 Redpoint 98 Flash 88 Follow 85 Pinkpoint 3 Solo 2 Attempt 1 Send 0 dtype: int64,29.600000,0.012323,1.869641,581,0.241882,3.750517
815,Unknown 5.8,Habitado > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/116668323/unknown-58,2.4,NaN,Trad,5.8+,5.8+,1,40.0,...,0.633987,1.045045,1.785714,Lead 187 Onsight 82 TR 77 Fell/Hung 37 Redpoint 19 Follow 17 Flash 15 Solo 1 Pinkpoint 0 Send 0 Attempt 0 dtype: int64,56.000000,0.166172,1.793561,116,0.344214,3.597900
354,Blue Sun,Way Rambo > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/105718435/blue-sun,3.6,NaN,Trad,5.10-,5.10-,1,70.0,...,0.725537,1.070465,1.557143,Lead 1010 Onsight 474 TR 371 Flash 134 Fell/Hung 119 Redpoint 108 Follow 51 Pinkpoint 2 Solo 1 Attempt 1 Send 0 dtype: int64,18.000000,0.008249,1.646315,714,0.327223,3.302524
819,Mariposa,Habitado > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/116668339/mariposa,2.7,NaN,Trad,5.10-,5.10-,1,50.0,...,0.622642,1.054054,2.000000,TR 117 Lead 60 Onsight 27 Fell/Hung 14 Flash 6 Redpoint 6 Solo 0 Follow 0 Pinkpoint 0 Send 0 Attempt 0 dtype: int64,31.000000,0.146226,1.624408,39,0.183962,3.258577
1,Binou's Crack,Donnelly Canyon > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/105717346/binous-crack,2.7,NaN,Trad,5.8+,5.8+,1,70.0,...,0.784281,1.038536,1.557377,Lead 752 Onsight 393 TR 332 Flash 76 Follow 70 Redpoint 63 Fell/Hung 56 Pinkpoint 10 Solo 1 Send 0 Attempt 0 dtype: int64,22.200000,0.011467,1.434985,539,0.278409,2.878593
816,Lightning Bolt Crack,Habitado > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/116177998/lightning-bolt-crack,3.1,NaN,Trad,5.9,5.9,1,50.0,...,0.631356,1.060976,1.562500,Lead 288 TR 133 Onsight 120 Fell/Hung 62 Flash 29 Redpoint 20 Follow 17 Pinkpoint 5 Solo 0 Send 0 Attempt 0 dtype: int64,60.250000,0.119307,1.418233,174,0.344554,2.844989
141,Twin Cracks,Supercrack Buttress > Indian Creek > Southeast Utah > Utah,https://www.mountainproject.com/route/105717445/twin-cracks,2.5,NaN,Trad,5.8+,5.8+,1,40.0,...,0.810241,1.050590,1.645161,Lead 824 Onsight 436 TR 288 Flash 102 Redpoint 80 Follow 59 Fell/Hung 40 Pinkpoint 6 Solo 3 Send 0 Attempt 0 dtype: int64,21.600000,0.011676,1.382420,623,0.336757,2.773147


In [355]:
fig = px.histogram(zscore(df['C Score'], nan_policy='omit'), nbins=50, marginal='box', title='<b>Z Score of Cumulative Metric</b><br>Number of Ticks > 100')
fig.update_layout(showlegend=False, title_x=0.5)
fig.update_xaxes(title="Z Score of Cumulative Metric", row=1,col=1, range=_)
fig.update_yaxes(title='Count', row=1,col=1)
fig.write_html("Cum_Hist.html")
fig